In [70]:
import torch
# a = torch.arange(.5,1000.5,1.).reshape(100, 10).type(torch.tensor)
a = torch.arange(.5,1000.5,1.).reshape(100, 10)
print(a)

tensor([[5.0000e-01, 1.5000e+00, 2.5000e+00, 3.5000e+00, 4.5000e+00, 5.5000e+00,
         6.5000e+00, 7.5000e+00, 8.5000e+00, 9.5000e+00],
        [1.0500e+01, 1.1500e+01, 1.2500e+01, 1.3500e+01, 1.4500e+01, 1.5500e+01,
         1.6500e+01, 1.7500e+01, 1.8500e+01, 1.9500e+01],
        [2.0500e+01, 2.1500e+01, 2.2500e+01, 2.3500e+01, 2.4500e+01, 2.5500e+01,
         2.6500e+01, 2.7500e+01, 2.8500e+01, 2.9500e+01],
        [3.0500e+01, 3.1500e+01, 3.2500e+01, 3.3500e+01, 3.4500e+01, 3.5500e+01,
         3.6500e+01, 3.7500e+01, 3.8500e+01, 3.9500e+01],
        [4.0500e+01, 4.1500e+01, 4.2500e+01, 4.3500e+01, 4.4500e+01, 4.5500e+01,
         4.6500e+01, 4.7500e+01, 4.8500e+01, 4.9500e+01],
        [5.0500e+01, 5.1500e+01, 5.2500e+01, 5.3500e+01, 5.4500e+01, 5.5500e+01,
         5.6500e+01, 5.7500e+01, 5.8500e+01, 5.9500e+01],
        [6.0500e+01, 6.1500e+01, 6.2500e+01, 6.3500e+01, 6.4500e+01, 6.5500e+01,
         6.6500e+01, 6.7500e+01, 6.8500e+01, 6.9500e+01],
        [7.0500e+01, 7.1500

In [ ]:
def softmax(x): return x.exp() / (x.exp().sum(-1)).unsqueeze(-1)
def nl(input, target): return -input[range(target.shape[0]), target].log().mean()
# target: (100), input: (100,10)



In [ ]:
b = torch.arange(10.,110.).type(torch.tensor)
print(b.shape)
print(a.shape)

In [ ]:
smf = softmax(a)
loss = nl(smf, b)


In [1]:
import torch
import torchvision
import os 
from model import CSRNet
from student_model import CSRNet_student

net = CSRNet().cuda()

# model_path ='best_model/mb2-ssd-lite-Epoch-14-Loss-1.7607919469712272.pth'
# net.load(model_path)
# net.eval()
# net.to('cuda')

# model_name = (model_path.split('/')[-1]).replace('.pth', '')
# model_path = f"onnx_model/{model_name}.onnx"
# if not os.path.exists(model_path.split('/')[0]):
#     os.mkdir(model_path.split('/')[0])
# # 1,2,300,300 -> 1:image 3:shape 300: height 300 width

model_path = "teacher_architechture.onnx"
dummy_input = torch.randn(1, 3, 170, 255, device='cuda')
torch.onnx.export(net, dummy_input, model_path, verbose=False, input_names=['input'], output_names=['t_out', 't_kd_list', 't_resize_list'])
